In [1]:
import os
import numpy as np
import random
import csv
import warnings
warnings.filterwarnings('ignore')

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs

from smiles_util import *
%load_ext autoreload
%autoreload 2

In [2]:
# SMILES data
path_data_smiles = './data/chembl_22_canonized.smi' 

In [3]:
# Read in the smiles
smiles_list = read_smiles_file(path_data_smiles)

In [4]:
# Make them canonical 
#canonized_smile_list = canonize_smiles(smiles_list) # We have already preprocessed the similes 
canonized_smile_list = smiles_list

In [5]:
# Find only unique SMILES
canonized_smile_list = list(set(canonized_smile_list))
canonized_smile_list = canonized_smile_list[1:]

In [6]:
from smileGenerator import smileGeneratorLSTM

Using TensorFlow backend.


In [9]:
# Generator #weights_path = 'model_lstm_smile_weights_maxlen100.h5'
gen = smileGeneratorLSTM(checkpoint_path = 'checkpoint_lstm_smile_maxlen100.h5') 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 100, 200)          9000      
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 200)          0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 100, 512)          935936    
_________________________________________________________________
attention (AttentionWeighted (None, 512)               512       
_________________________________________________________________
dense_2 (Dense)              (None, 45)                23085     
Total params: 968,533
Trainable params: 968,533
Non-trainable params: 0
_________________________________________________________________


In [49]:
# Train Generator to learn SMILES
gen.train(smiles_list, batch_size = 1280, epochs = 20, validation_split = 0.1)

Train on 90000 samples, validate on 10000 samples
Epoch 1/20
90000/90000 [==============================] - 1180s 13ms/step - loss: 0.8633 - acc: 0.7050 - val_loss: 0.8535 - val_acc: 0.7101

Epoch 00001: val_loss improved from inf to 0.85346, saving model to checkpoint_lstm_smile_maxlen100.h5
Epoch 2/20
90000/90000 [==============================] - 1166s 13ms/step - loss: 0.8402 - acc: 0.7131 - val_loss: 0.8543 - val_acc: 0.7091

Epoch 00002: val_loss did not improve from 0.85346
Epoch 3/20
90000/90000 [==============================] - 1176s 13ms/step - loss: 0.8218 - acc: 0.7187 - val_loss: 0.8480 - val_acc: 0.7107

Epoch 00003: val_loss improved from 0.85346 to 0.84796, saving model to checkpoint_lstm_smile_maxlen100.h5
Epoch 4/20
90000/90000 [==============================] - 1168s 13ms/step - loss: 0.8069 - acc: 0.7228 - val_loss: 0.8524 - val_acc: 0.7087

Epoch 00004: val_loss did not improve from 0.84796
Epoch 5/20
90000/90000 [==============================] - 1165s 13ms/step 

In [81]:
# Save model
gen.save_model(weights_path = 'model_lstm_smile_weights_maxlen100.h5')

In [76]:
# Load model
# gen.load_model(weights_path = 'model_lstm_smile_weights.h5')
from keras.models import load_model
from AttentionWeightedAverage import AttentionWeightedAverage
gen_best_model = load_model('checkpoint_lstm_smile_maxlen100.h5', 
                                 custom_objects = {'AttentionWeightedAverage':AttentionWeightedAverage})

In [77]:
#gen_best_model.summary()

In [78]:
# Sample SMILES from generator
cs = gen.generate_smiles_list(100)

SMILES generated: 100, SMILES valid: 30
